In [ ]:
import numpy as np
import matplotlib.pylab as plt
#from scipy.misc import imresize, imread
from PIL import Image
import itertools
import sklearn
from sklearn import model_selection
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold, learning_curve, GridSearchCV
from sklearn.metrics import confusion_matrix, make_scorer, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import keras
import glob
import os
import shutil
"""
from keras import backend as K
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, model_from_json
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Conv2D, MaxPool2D, MaxPooling2D
"""

%matplotlib inline

In [ ]:
pip install --upgrade tensorflow

In [ ]:
pip install tensorflow==2.0

### Organiser les images

Réorganiser les fichiers dans 0/1 dosiers et éliminer les dossers numéro patient

In [ ]:
def organize_images(data_path, pre_processed_data_path):
    files = glob.glob(data_path + '\**\*.png', recursive=True)
    class_0_path = os.path.join(pre_processed_data_path,"0")
    class_1_path = os.path.join(pre_processed_data_path,"1")
    os.makedirs(class_0_path,exist_ok=True)
    os.makedirs(class_1_path,exist_ok=True)
    
    for file in files:
        if "class0" in file:
            file_dest = os.path.join(class_0_path,os.path.basename(file))
        elif "class1" in file:  
            file_dest = os.path.join(class_1_path,os.path.basename(file))
        shutil.copyfile(file, file_dest)

organize_images("..\..\data", "..\..\pre_processed_data")

In [ ]:
xx = glob.glob('..\..\data\**\*.png', recursive=True)
print(len(unique(xx)))

In [ ]:
"""
def unique(list1): 
  
    # intilize a null list 
    unique_list = [] 
      
    # traverse for all elements 
    for x in list1: 
        # check if exists in unique_list or not 
        if x not in unique_list: 
            unique_list.append(x) 
    # print list 
    #for x in unique_list: 
    #    print x,
    return unique_list
"""